# balanced network

Let's start with a rather complete example modified from 

http://neuralensemble.org/docs/PyNN/examples/VAbenchmarks.html


In [1]:
%run 3_boilerplate.ipynb

CSAConnector: libneurosim support not available in NEST.
Falling back on PyNN's default CSAConnector.
Please re-compile NEST using --with-libneurosim=PATH


/home/laurent/.local/lib/python3.8/site-packages/pyNN/nest/__init__.py:53: UserWarning:Unable to install NEST extensions. Certain models may not be available.


Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.18.1

numpy     : 1.19.2
matplotlib: 3.3.2
pyNN      : 0.9.5
neo       : 0.8.0
nixio     : 1.5.0b6

Compiler    : GCC 9.3.0
OS          : Linux
Release     : 5.4.0-56-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 4
Architecture: 64bit

Hostname: inv-ope-de06

Git hash: f30070d2bccb0987fe0c715827e293af15674169

Git repo: https://github.com/SpikeAI/2020-11_brainhack_Project7

Git branch: main



## running it...

In [2]:
run(do_nixio=True, do_hdf5=True)

/usr/lib/python3.8/dist-packages/nest/lib/hl_api_helper.py:127: UserWarning:
ResetNetwork is deprecated and will be removed in NEST 3.0.
2020-12-07 14:20:10,526 WARNING  [PyNN] NEST does not allow setting an initial value for g_ex (/home/laurent/.local/lib/python3.8/site-packages/pyNN/nest/populations.py[174]:_set_initial_value_array)
2020-12-07 14:20:10,528 WARNING  [PyNN] NEST does not allow setting an initial value for g_in (/home/laurent/.local/lib/python3.8/site-packages/pyNN/nest/populations.py[174]:_set_initial_value_array)


0 Creating cell populations...
0 Initialising membrane potential to random values...
0 Connecting populations...
0 Setting up recording...0%                                                                                                                                                                                                                                                                                                 
0 Running simulation...
0 Writing data to file...


AssertionError: This type is not supported by this file format for writing

to import nixio and get help, run

    import nixio
    help(nixio)

on spinnaker, run 

    pip install -U neo[nixio]
    
or  

    pip install -U nixio
    
    
TODO: read and checkout what's in Results/20201202/VAbenchmarks_inh_nest_20201202-172001.h5

In [ ]:
#pip list

In [ ]:
ls